In [2]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns

In [3]:
## Files are the plot to atm temperature file once it has been modified for this notebook and the csv that has image and plot
env_logger = pd.read_csv('full_image_file_mod.csv').set_index('Image Name')
img_plot = pd.read_csv('finale_df_mod.csv')

In [4]:
del img_plot['Unnamed: 0']

In [5]:
## Renames the columns so it works with Emmanuel's code
img_plot.columns = ['image', 'plot']

In [8]:
## Adds all of the AZmet and Env data to the file that just had image name and plot (essentially merges them together)
img_plot['Date and Time'] = img_plot['azmet atm temp'] = img_plot['azmet wind velocity'] = img_plot['azmet VPD'] = img_plot['azmet solar radiation'] = img_plot['relative humidity'] = img_plot['Env Temperature'] = img_plot['Env Wind Velocity'] = None

for i, row in img_plot.iterrows():
    meta = row['image']
    try:
        #print(env_logger.loc[meta])
        datetime = env_logger.loc[meta, 'Date and Time']
        azmet_temp = env_logger.loc[meta, 'azmet atm temp']
        azmet_wind_vel = env_logger.loc[meta, 'azmet wind velocity']
        azmet_vpd = env_logger.loc[meta, 'azmet VPD']
        sol_rad = env_logger.loc[meta, 'azmet solar radiation']
        temp = env_logger.loc[meta, 'Env Temperature']
        win_vel = env_logger.loc[meta, 'Env Wind Velocity']
        rel_hum = env_logger.loc[meta, 'relative humidity']
        #print(sun_dir)
        
        img_plot.at[i, 'Date and Time'] = datetime
        img_plot.at[i, 'azmet atm temp'] = azmet_temp
        img_plot.at[i, 'azmet wind velocity'] = azmet_wind_vel
        img_plot.at[i, 'azmet VPD'] = azmet_vpd
        img_plot.at[i, 'azmet solar radiation'] = sol_rad
        img_plot.at[i, 'Env Temperature'] = temp
        img_plot.at[i, 'Env Wind Velocity'] = win_vel
        img_plot.at[i, 'relative humidity'] = rel_hum
        
    except:
        pass
    

In [9]:
## Drops image and time columns so we can get stats on the values
df_agg = img_plot.drop(['image', 'Date and Time'], axis=1)#.set_index('plot')#.groupby('plot')

In [11]:
## Creates a dictionary that goes through the merged dataframe and calculates different statistical values for the data
temp_dict = {}
cnt = 0

for plot in df_agg['plot'].unique().tolist():
    try:
        cnt += 1 

        select_df = df_agg.set_index('plot').loc[plot]
        temp_median = select_df['azmet atm temp'].median()
        temp_mean = select_df['azmet atm temp'].mean()
        temp_std = select_df['azmet atm temp'].std()
        
        azmet_wind_vel = select_df['azmet wind velocity'].median()
        azmet_vpd = select_df['azmet VPD'].median()
        sol_rad = select_df['azmet solar radiation'].median()
        temp = select_df['Env Temperature'].median()
        wind_vel = select_df['Env Wind Velocity'].median()
        rel_hum = select_df['relative humidity'].median()
        
        
        
        
        #print(temp_median)
        temp_dict[cnt] = {'plot': plot,
                          'median': temp_median,
                          'mean': temp_mean, 
                          'std_dev': temp_std, 
                          'azmet wind velocity': azmet_wind_vel, 
                          'azmet VPD': azmet_vpd, 
                          'azmet solar radiation': sol_rad, 
                          'Env Temperature': temp, 
                          'wind velocity': wind_vel,
                         'relative humidity': rel_hum}
    except:
        pass

In [12]:
## Converts dictionary with stats into a dataframe and sets the plot as the index
result = pd.DataFrame.from_dict(temp_dict, orient='index').set_index('plot')

In [15]:
## Reads in the csv with the individual plant temperatures (produced by the pipeline)
plant_detections = pd.read_csv('2020-03-03_individual_temps.csv')

In [16]:
## Adds the field information and PCT values to the already existing csv that is indexed by plot
plant_detections['norm_temp'] = plant_detections['atm_temp'] = plant_detections['relative_humidity'] = None

for i, row in plant_detections.iterrows():

    try:
        plot = row['plot'].replace('_', ' ')
        plant_temp = row['median']
        
        temp_df = result.loc[plot]
        atm_temp = temp_df['median']
        norm_temp =  atm_temp - plant_temp
        #norm_temp = plant_temp - atm_temp
        
        azmet_wind_vel = temp_df['azmet wind velocity']
        azmet_vpd = temp_df['azmet VPD']
        sol_rad = temp_df['azmet solar radiation']
        temp = temp_df['Env Temperature']
        wind_vel = temp_df['wind velocity']
        rel_hum = temp_df['relative humidity']
        
        plant_detections.at[i, 'norm_temp'] = norm_temp
        plant_detections.at[i, 'atm_temp'] = atm_temp
        
        plant_detections.at[i, 'azmet wind velocity'] = azmet_wind_vel
        plant_detections.at[i, 'azmet VPD'] = azmet_vpd
        plant_detections.at[i, 'azmet solar radiation'] = sol_rad
        plant_detections.at[i, 'Env Temperature'] = temp
        plant_detections.at[i, 'wind velocity'] = wind_vel
        plant_detections.at[i, 'relative humidity'] = rel_hum
        #print(atm_temp)
    except:
        pass

In [18]:
plant_detections.to_csv('plant_detections_new.csv')

In [20]:
plant_detections.to_csv('/mnt/c/Users/emman/OneDrive - University of Arizona/work/data/season_10/flir_env_exp/thermal_kmeans_approach_figs/2020-03-03_individual_temps_env_logger.csv', index=False)

In [26]:
plant_detections.to_csv('/mnt/c/Users/emman/Downloads/season10_2020-03-03_ctd.csv')